In [39]:
import math

def count_repetitions(ciphertext):
    blocks = [ciphertext[i:i+16] for i in range(0, len(ciphertext), 16)]
    total_blocks = len(blocks)
    unique_blocks = len(set(blocks))
    return total_blocks - unique_blocks


def pad(block_size : int , text : str) -> str:
    pad_value = block_size - len(text) % block_size
    return text + chr(pad_value) * pad_value

text = "YELLOW SUBMARINE"
print("Padded:- " , pad(16 , text))

Padded:-  YELLOW SUBMARINE


### CBC decrypt AES

In [40]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
import base64

def byte_xor(ba1, ba2):
    return bytes([_a ^ _b for _a, _b in zip(ba1, ba2)])

def aes_ecb_decrypt(ct , key):
    cipher = AES.new(key , AES.MODE_ECB)
    return cipher.decrypt(ct)

def aes_cbc_decrypt(ct , key , IV):
    blocks = [ct[i:i+16] for i in range(0, len(ct), 16)]
    plaintext = []
    for i,block in enumerate(blocks):
        ith_block = aes_ecb_decrypt(block , key)
        if i == 0:
            pt = byte_xor(ith_block , IV)
        else:
            pt = byte_xor(ith_block , blocks[i-1])
        plaintext.append(pt)
    return b"".join(plaintext)


file_path = '/run/media/biprarshi/COMMON/files/Crypto/CryptoPals/data/set2q10.txt'
with open(file_path, 'r', ) as f:
    ciphertext = base64.b64decode(f.read())

key = b"YELLOW SUBMARINE"
IV = b'\x00' * 16
print("Plaintext")
print(aes_cbc_decrypt(ciphertext , key, IV).decode('utf-8'))

Plaintext
I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make 

In [41]:
from random import randint
from Crypto.Cipher.AES import block_size
from Crypto import Random
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad

BLOCK_SIZE = 16


def byte_xor(a, b):
    return bytes(x ^ y for x, y in zip(a, b))

def aes_ecb_encrypt(data, key):
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.encrypt(data)

def aes_cbc_encrypt(pt, key, iv):
    pt = pad(pt, BLOCK_SIZE)
    cipher = AES.new(key, AES.MODE_ECB)

    blocks = [pt[i:i+BLOCK_SIZE] for i in range(0, len(pt), BLOCK_SIZE)]
    ciphertext = []

    prev = iv
    for block in blocks:
        xored = byte_xor(block, prev)
        encrypted = cipher.encrypt(xored)
        ciphertext.append(encrypted)
        prev = encrypted

    return b''.join(ciphertext)

class AesEncryptOracle:
    @staticmethod
    def random_pad(bin):
        return Random.new().read(randint(5, 10)) + bin + Random.new().read(randint(5, 10))
    @staticmethod
    def encrypt(pt):
        padded_plaintext = AesEncryptOracle.random_pad(pt)
        key = Random.new().read(BLOCK_SIZE)

        if randint(0, 1) == 0:
            return "ECB", aes_ecb_encrypt(
                pad(padded_plaintext, BLOCK_SIZE), key
            )
        else:
            return "CBC", aes_cbc_encrypt(
                padded_plaintext, key, Random.new().read(BLOCK_SIZE)
            )

    @staticmethod
    def count_repetitions(ciphertext):
        blocks = [
            ciphertext[i:i+BLOCK_SIZE]
            for i in range(0, len(ciphertext), BLOCK_SIZE)
        ]
        return len(blocks) - len(set(blocks))

    @staticmethod
    def detect_cipher(ciphertext):
        if AesEncryptOracle.count_repetitions(ciphertext) > 0:
            return "ECB"
        else:
            return "CBC"

oracle = AesEncryptOracle()
input_data = bytes([0]*64)
for _ in range(1000):
        encryption_used, ciphertext = AesEncryptOracle.encrypt(input_data)
        encryption_detected = AesEncryptOracle.detect_cipher(ciphertext)
        assert encryption_used == encryption_detected
print("All tests passed.")

All tests passed.


## Byte-at-a-time ECB decryption (Simple)

In [42]:
from base64 import b64decode
from Crypto import Random
from Crypto.Cipher import AES


def aes_ecb_encrypt(data, key):
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.encrypt(data)

class ECBOracle:
    def __init__(self , secret_padding):
        self.key = Random.new().read(AES.key_size[0])
        self.secret_pad = secret_padding

    def encrypt(self , data):
        pt = pad(data + self.secret_pad , BLOCK_SIZE)
        return aes_ecb_encrypt(pt, self.key)


def detect_block_length(enc_oracle):
    base_len = len(enc_oracle.encrypt(b""))
    i = 1
    while True:
        new_len = len(enc_oracle.encrypt(b"A" * i))
        if new_len > base_len:
            return new_len - base_len
        i += 1

def get_next_byte(block_len, known_pad, enc_oracle : ECBOracle):
    len_to_use = (block_len - (1 + len(known_pad))) % block_len
    prefix = b'A' * len_to_use

    block_index = len(known_pad) // block_len
    real_ct = enc_oracle.encrypt(prefix)
    target_block = real_ct[block_index*block_len:(block_index+1)*block_len]

    for i in range(256):
        fake_ct = enc_oracle.encrypt(prefix + known_pad + bytes([i]))
        guess_block = fake_ct[block_index*block_len:(block_index+1)*block_len]
        if guess_block == target_block:
            return bytes([i])
    print("bad")
    return None ## shouldn't run


def per_byte_ecb_dec(enc_oracle : ECBOracle):
    block_len = detect_block_length(enc_oracle)
    ciphertext = enc_oracle.encrypt(bytes([0]) * 64)
    assert count_repetitions(ciphertext) > 0

    base_len = len(enc_oracle.encrypt(b''))

    i = 1
    while True:
        new_len = len(enc_oracle.encrypt(b"A" * i))
        if new_len > base_len:
            padding_trigger = i
            break
        i += 1

    total_len = base_len - padding_trigger

    recovered = b""
    for _ in range(total_len):
        next_byte = get_next_byte(block_len, recovered, oracle)
        if next_byte is None:
            break
        recovered += next_byte

    return recovered

secret_padding = b64decode("Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGF"
                               "pciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IH"
                               "RvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK")
oracle = ECBOracle(secret_padding)
dec_secret_pad = per_byte_ecb_dec(oracle)
assert dec_secret_pad == secret_padding
print("Recovered successfully!")
print(dec_secret_pad.decode('utf-8'))


Recovered successfully!
Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



## Challenge 13

In [44]:
import re
from Crypto.Util.Padding import pad, unpad

def profile_for(email):
    email = email.replace('&', '').replace('=', '')     # Remove special characters to avoid injection
    return {
        'email': email,
        'uid': 10,
        'role': 'user'
    }

def kv_encode(dict_object):
    encoded_text = ''
    for item in dict_object.items():
        encoded_text += item[0] + '=' + str(item[1]) + '&'
    return encoded_text[:-1] # without the last '&' character


def kv_parse(encoded_text):
    output = {}
    attributes = encoded_text.split('&')

    for attribute in attributes:
        values = attribute.split('=')
        key = int(values[0]) if values[0].isdigit() else values[0]
        value = int(values[1]) if values[1].isdigit() else values[1]
        output[key] = value

    return output

class ECBOracleEmail(object):
    def __init__(self):
        self.key = Random.new().read(AES.key_size[0])

    def encrypt(self, email):
        encoded = kv_encode(profile_for(email))
        pt_bytes = pad(encoded.encode(),BLOCK_SIZE)
        cipher = AES.new(self.key , AES.MODE_ECB)
        return cipher.encrypt(pt_bytes)

    def decrypt(self, ct):
        cipher = AES.new(self.key, AES.MODE_ECB)
        pt = unpad(cipher.decrypt(ct), 16)
        return kv_parse(pt.decode())

def ecb_cut_paste(oracle : ECBOracleEmail):
    prefix_len = BLOCK_SIZE - len("email=")
    suffix_len = BLOCK_SIZE - len("admin")
    email1 = "x" * prefix_len + "admin" + (chr(suffix_len) * suffix_len) ## make synthetic email with email=xxxxxxx + admin_randomjunk block (0-32)
    encrypted_email1 = oracle.encrypt(email1)

    email2 = "master@me.com"
    encrypted_email2 = oracle.encrypt(email2)

    injection = encrypted_email2[:32] + encrypted_email1[16:32] ##  email=master@me.   com&uid=10&role=   admin\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b

    return injection


oracle = ECBOracleEmail()
injection = ecb_cut_paste(oracle)
parsed = oracle.decrypt(injection)
print(parsed)


{'email': 'master@me.com', 'uid': 10, 'role': 'admin'}
